### The libraries needed

In [2]:
import pandas as pd
import numpy as np
from scipy import stats
import chardet

Finding the encoding of the dataset

In [3]:

with open("dataset.csv", "rb") as f:
    result = chardet.detect(f.read(100000))  # Read a chunk of the file
    print(result["encoding"])  # Print the detected encoding


Windows-1252


Loading the dataset but only some of the columns we need 


In [4]:
df=pd.read_csv("dataset.csv", encoding="Windows-1252", sep=";",usecols=[0,1,2,5,6,7,10,11,13,14,17])
df.head(100)

,id,category,title,bathrooms,bedrooms,currency,pets_allowed,price,price_type,square_feet,state
0,5668640009,housing/rent/apartment,One BR 507 & 509 Esplanade,1.0,1.0,USD,Cats,2195.0,Monthly,542,CA
1,5668639818,housing/rent/apartment,Three BR 146 Lochview Drive,1.5,3.0,USD,"Cats,Dogs",1250.0,Monthly,1500,VA
2,5668639686,housing/rent/apartment,Three BR 3101 Morningside Drive,2.0,3.0,USD,NaN,1395.0,Monthly,1650,NC
3,5668639659,housing/rent/apartment,Two BR 209 Aegean Way,1.0,2.0,USD,"Cats,Dogs",1600.0,Monthly,820,CA
4,5668639374,housing/rent/apartment,One BR 4805 Marquette NE,1.0,1.0,USD,"Cats,Dogs",975.0,Monthly,624,NM
...,...,...,...,...,...,...,...,...,...,...,...
95,5668633801,housing/rent/apartment,Two BR 1917 S. 18th St.,1.0,2.0,USD,"Cats,Dogs",1015.0,Monthly,845,NE
96,5668632658,housing/rent/apartment,Three BR 7312 South 81st Street,2.0,3.0,USD,"Cats,Dogs",1495.0,Monthly,1850,NE
97,5668632537,housing/rent/apartment,One BR 4301 Grand Avenue Parkway,1.0,1.0,USD,NaN,1103.0,Monthly,652,TX
98,5668632393,housing/rent/apartment,One BR 2101 W. ANDERSON LN.,1.0,1.0,USD,NaN,1032.0,Monthly,600,TX


It seems like we have mostly monthly bills and 3 weekly bills we will make it even by multiplying the price with 4 to make it monthly, as for the monthly|weekly part i will just drop it.

In [5]:
df["price_type"].value_counts()

price_type
Monthly           99488
Weekly                3
Monthly|Weekly        1
Name: count, dtype: int64

In [6]:
df["price"]=df.apply( lambda row: row["price"]*4 if row["price_type"]=="Weekly" else row["price"],axis=1)
df["price_type"]=df.apply( lambda row: "Monthly" if row["price_type"]=="Weekly" else row["price_type"],axis=1)
df=df[df["price_type"]!= "Monthly|Weekly"]

Lets drop the NaN values for the important variables

In [7]:
df = df.dropna(subset=["price"])

In [8]:
df["square_feet"].isna().value_counts()
#it doesnt have nan values so we will not perform anything

square_feet
False    99490
Name: count, dtype: int64

Since there are a lot of data i will use the data for 4 major states: New York, Texas, California and Florida. So i will filter those:

In [9]:
df_ny=df[df["state"]=="NY"]#Dataframe for new york
df_tx=df[df["state"]=="TX"]#Dataframe for Texas
df_ca=df[df["state"]=="CA"]#Dataframe for California
df_fl=df[df["state"]=="FL"]#Dataframe for Florida


### What I wanna do is make a linear regression where I take the square feet of the property as the independent variable and for the dependent variable to be the price for each of the states

In [10]:
#x will be the independent variable
x_ny=df_ny['square_feet'].values.tolist()
x_tx=df_tx['square_feet'].values.tolist()
x_ca=df_ca['square_feet'].values.tolist()
x_fl=df_fl['square_feet'].values.tolist()
#y will be the dependent variable
y_ny=df_ny['price'].values.tolist()
y_tx=df_tx['price'].values.tolist()
y_ca=df_ca['price'].values.tolist()
y_fl=df_fl['price'].values.tolist()

Now creating the Linear regression stats

In [11]:
slope_ny, intercept_ny, r_ny, p_ny, std_err_ny = stats.linregress(x_ny, y_ny)
slope_tx, intercept_tx, r_tx, p_tx, std_err_tx = stats.linregress(x_tx, y_tx)
slope_ca, intercept_ca, r_ca, p_ca, std_err_ca = stats.linregress(x_ca, y_ca)
slope_fl, intercept_fl, r_fl, p_fl, std_err_fl = stats.linregress(x_fl, y_fl)

The functions of the Regression lines will be:

In [12]:
def ref_func_ny(x):
    return slope_ny*x+intercept_ny

def ref_func_tx(x):
    return slope_tx*x+intercept_tx

def ref_func_ca(x):
    return slope_ca*x+intercept_ca

def ref_func_fl(x):
    return slope_fl*x+intercept_fl

Lets print the important findings that will be used in Power BI visualisations

In [13]:
print(slope_ny,intercept_ny,p_ny,std_err_ny,r_ny*r_ny)
print(slope_tx,intercept_tx,p_tx,std_err_tx,r_tx*r_tx)
print(slope_ca,intercept_ca,p_ca,std_err_ca,r_ca*r_ca)
print(slope_fl,intercept_fl,p_fl,std_err_fl,r_fl*r_fl)

0.2182635082421261 2377.9517777205097 2.4388301279374208e-05 0.05134658375275616 0.02676645493963366
0.7841285775092612 508.9743419683962 0.0 0.00944223002037154 0.37993924610005547
2.2569284293494105 374.58292819466715 0.0 0.0291989120513961 0.36690550714187725
1.3770875899050876 137.21566033206113 0.0 0.026814904642868426 0.3136598578263035


In [14]:
print(r_ny,r_tx,r_ca,r_fl)

0.16360456882261468 0.6163921204071767 0.605727254745795 0.5600534419377347


Lets save the cleaned dataset so we can use it in Power BI

In [15]:
df.to_csv("cleaned_dataset.csv")

In [16]:
correlation= df_fl["square_feet"].corr(df_fl["price"])
correlation

np.float64(0.5600534419377347)

In [18]:
df["pets_allowed"].value_counts()

pets_allowed
Cats,Dogs         37095
Cats               1843
Dogs                127
Cats,Dogs,None        1
Name: count, dtype: int64

In [ ]:
df2=pd.read_csv("US States Ranked by Population 2024.csv")
df2

,Rank,US State,Population 2024,Population 2023,Growth Rate,% of US,Density (/mile2)
0,1,California,38889770,38965193,-0.0019,0.1158,250
1,2,Texas,30976754,30503301,0.0155,0.0922,119
2,3,Florida,22975931,22610726,0.0162,0.0684,428
3,4,New York,19469232,19571216,-0.0052,0.0580,413
4,5,Pennsylvania,12951275,12961683,-0.0008,0.0386,289
5,6,Illinois,12516863,12549689,-0.0026,0.0373,225
6,7,Ohio,11812173,11785935,0.0022,0.0352,289
7,8,Georgia,11145304,11029227,0.0105,0.0332,194
8,9,North Carolina,10975017,10835491,0.0129,0.0327,226
9,10,Michigan,10041241,10037261,0.0004,0.0299,178


In [23]:
X=df2["Density (/mile2)"].values.tolist()

In [29]:
Y=df.groupby("state")["price"].mean()
Y.values.tolist()

[1050.7586206896551,
 959.8022598870057,
 874.3678929765887,
 1119.4922301553968,
 2463.468140820483,
 1554.123211446741,
 1266.2298624754421,
 2112.1397849462364,
 1154.857142857143,
 1574.0168023557942,
 1351.296085858586,
 2729.1935483870966,
 949.5268817204301,
 1122.3229166666667,
 1483.2432432432433,
 1011.9076620825148,
 918.3508771929825,
 989.2834070351759,
 1026.063940520446,
 2192.386679920477,
 1645.4575757575758,
 1247.65625,
 1196.7957746478874,
 1391.9363166953528,
 1048.4305901911887,
 874.1588785046729,
 1118.344827586207,
 1153.834761904762,
 945.4710632570659,
 933.163725490196,
 1453.174149659864,
 2037.9145106861642,
 844.0416666666666,
 1173.4021314387212,
 2598.1289833080423,
 1110.727013251784,
 1015.3190578158458,
 1475.3574007220216,
 1301.3244206773618,
 2034.7310924369747,
 1143.715859030837,
 887.6511627906976,
 1177.4887892376682,
 1217.6280536555032,
 1300.6514215080347,
 1485.8416224046355,
 1415.08,
 1825.7429672447013,
 1230.7,
 871.7692307692307,
 791